<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=deb5cda8e247ecc48dc82a04ce69675f752525060522c8cf6dbd230d0ec886cf
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 14:21:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -71.59 K (-0.51%)
Current PnL: -26.67 L (-17.23%)
CY Booked + Current PnL: -12.38 L (-8.0%)
-------------------
Total profit:  1.37 L
Total loss:  -28.04 L
-------------------
Total Booked + Current PnL: 14.60 L (11.42%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.30 L (65.04%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.22%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.90,-18.03,22.12,0.09,18272.0,-18175.0,82602.0,91.68,35.0,M-SC,2.34,253.0,-0.99,0.59,8.52,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.32,9.01,9.80,19.69,20483.0,17268.0,209010.0,-1.63,58.0,X-MC,2.53,78.0,0.84,1.50,16.42,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.83,-13.29,15.35,0.02,23233.0,-23205.0,151354.0,131.03,49.0,M-SC,14.79,234.0,-1.00,1.08,31.65,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.08,-2.11,24.68,22.05,23671.0,-2070.0,95910.0,-15.54,54.0,H-SC,7.25,164.0,-0.09,0.69,39.41,XR,ATH,FINANCE
43,ITC,452.00,-0.02,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.74,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,3.00,-30.24,77.60,23.89,95219.0,-53198.0,122705.0,-32.70,39.0,H-SC,10.58,157.0,-0.56,0.88,5.97,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.62,-1.45,110.97,107.91,161467.0,-2139.0,145505.0,-19.74,59.0,M-SC,12.53,216.0,-0.01,1.04,8.49,OX40N,ATH,CABLES
6,ASIANTILES,137.00,1.97,-16.88,118.57,81.67,92454.0,-15836.0,77974.0,7005.56,45.0,L-SC,12.36,271.0,-0.17,0.56,51.00,XR,NTT,CERAMICS
44,JCHAC,2282.00,1.66,-33.86,51.18,-0.01,43263.0,-43273.0,84532.0,17204.35,39.0,M-SC,7.32,233.0,-1.00,0.61,8.19,OX40N,NTT,AC
71,TANLA,1943.92,1.27,-35.62,241.22,119.69,420514.0,-96433.0,174328.0,-35.11,47.0,H-SC,7.99,156.0,-0.23,1.25,37.72,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.1,-3.33,-36.03,118.24,39.61,77530.0,-36930.0,65570.0,-56.80,34.0,H-SC,28.75,135.0,-0.48,0.47,3.31,AR,ATH,MISC
21,DEN,75.0,-2.31,-40.80,140.00,42.07,70000.0,-34464.0,50000.0,78.76,46.0,M-SC,7.81,235.0,-0.49,0.36,6.18,AR,NTT,ENTERTAINMENT
0,5PAISA,593.0,-2.22,-40.98,91.29,12.90,117728.0,-89536.0,128960.0,89.90,45.0,H-SC,12.02,161.0,-0.76,0.92,7.23,OX40N,NTT,FINANCE
10,BANDHANBNK,400.0,-2.15,-28.55,169.31,92.42,336978.0,-79529.0,199030.0,40.09,48.0,H-SC,8.96,170.0,-0.24,1.42,15.02,XY24,NTT,BANKS
63,SFL,1287.0,-2.10,-44.17,126.82,26.63,185651.0,-115829.0,146389.0,10.90,26.0,M-SC,11.67,239.0,-0.62,1.05,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.62,-1.45,110.97,107.91,161467.0,-2139.0,145505.0,-19.74,59.0,M-SC,12.53,216.0,-0.01,1.04,8.49,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.83,-13.29,15.35,0.02,23233.0,-23205.0,151354.0,131.03,49.0,M-SC,14.79,234.0,-1.00,1.08,31.65,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.90,-18.03,22.12,0.09,18272.0,-18175.0,82602.0,91.68,35.0,M-SC,2.34,253.0,-0.99,0.59,8.52,OX40N,NTT,DURABLES
66,SIS,528.00,0.05,-24.19,59.95,21.26,50464.0,-26856.0,84176.0,1986.01,50.0,H-SC,5.12,166.0,-0.53,0.60,13.73,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.34,-24.71,50.72,13.47,102977.0,-66636.0,203031.0,-69.09,36.0,H-SC,5.19,158.0,-0.65,1.45,3.24,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.97,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.8,63.0,M-LC,2.45,99.0,0.1,1.29,12.88,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.97,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.88,XR,NTT,IT
25,FINCABLES,1641.55,2.62,-1.45,110.97,107.91,161467.0,-2139.0,145505.0,-19.74,59.0,M-SC,12.53,216.0,-0.01,1.04,8.49,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.39,-3.96,115.25,106.72,136514.0,-4886.0,118450.0,-52.54,35.0,H-SC,2.99,139.0,-0.04,0.85,18.49,AR,ATH,MISC
1,ABB,7934.00,-0.62,-3.64,51.07,45.57,128745.0,-9523.0,252096.0,-38.07,57.0,H-MC,4.55,120.0,-0.07,1.80,7.27,AR,NTT,ELECTRICAL
37,IEX,219.00,-0.61,-3.41,53.88,48.63,104057.0,-6826.0,193128.0,-35.77,50.0,H-SC,14.04,136.0,-0.07,1.38,7.74,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.24,-18.14,89.93,55.47,179093.0,-44139.0,199147.0,-27.39,25.0,X-MC,12.93,64.0,-0.25,1.43,2.04,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-1.20,-9.89,42.16,28.11,76520.0,-19910.0,181500.0,-34.75,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.72,-29.36,74.09,22.98,118228.0,-66325.0,159573.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.36,XY24,NTT,AUTO
8,AWL,485.00,-1.06,-22.86,98.97,53.50,230383.0,-68970.0,232781.0,-62.61,31.0,X-MC,2.79,58.0,-0.30,1.67,3.39,XY24,NTT,FMCG
33,HINDUNILVR,2922.00,-0.49,-11.22,28.03,13.67,63973.0,-28833.0,228230.0,-17.78,32.0,X-LC,2.14,14.0,-0.45,1.63,7.74,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.50,-1.55,26.43,24.48,65536.0,-3894.0,247962.0,-81.32,43.0,X-SC,0.31,86.0,-0.06,1.77,21.23,X40N,NTT,MISC
43,ITC,452.0,-0.02,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.74,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.49,-11.22,28.03,13.67,63973.0,-28833.0,228230.0,-17.78,32.0,X-LC,2.14,14.0,-0.45,1.63,7.74,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.30,5.91,23.43,30.73,48888.0,11651.0,208656.0,-18.85,59.0,X-MC,2.26,75.0,0.24,1.49,19.44,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.35,6.33,13.58,20.77,33810.0,14826.0,248970.0,-2.93,57.0,X-MC,2.44,62.0,0.44,1.78,13.69,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-1.20,-9.89,42.16,28.11,76520.0,-19910.0,181500.0,-34.75,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
3,ACC,3906.00,-0.33,-25.54,120.45,64.15,213413.0,-60771.0,177180.0,-56.31,34.0,X-SC,2.60,82.0,-0.28,1.27,0.01,XY24,BTT,CEMENT
75,TMPV,600.00,-0.72,-29.36,74.09,22.98,118228.0,-66325.0,159573.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.36,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.10,-38.12,118.49,35.20,94344.0,-49048.0,79622.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.14,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.48,-47.90,192.32,52.31,145455.0,-69528.0,75632.0,-44.54,41.0,X-SC,6.93,91.0,-0.48,0.54,1.24,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.47,-29.32,102.10,42.84,46911.0,-19060.0,45946.0,-53.72,44.0,X-SC,35.52,83.0,-0.41,0.33,3.95,XY24,NTT,MISC
59,RELAXO,1176.00,-0.48,-47.90,192.32,52.31,145455.0,-69528.0,75632.0,-44.54,41.0,X-SC,6.93,91.0,-0.48,0.54,1.24,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.10,-38.12,118.49,35.20,94344.0,-49048.0,79622.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.14,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.77,-4.86,30.42,24.08,36740.0,-6174.0,120776.0,-12.55,32.0,X-SC,5.12,89.0,-0.17,0.86,13.80,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.35,-16.63,71.92,43.32,97653.0,-27092.0,135780.0,-28.31,37.0,X-SC,4.75,90.0,-0.28,0.97,4.25,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.94,-12.02,37.20,20.71,123794.0,-45458.0,332779.0,-24.27,58.0,X-LC,4.46,1.0,-0.37,2.38,11.19,X40,ATH,IT
41,INFY,1972.00,-0.97,8.32,23.93,34.24,81105.0,26038.0,338926.0,-15.65,59.0,X-LC,4.00,2.0,0.32,2.43,17.29,X40,NTT,IT
75,TMPV,600.00,-0.72,-29.36,74.09,22.98,118228.0,-66325.0,159573.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.36,XY24,NTT,AUTO
81,VBL,671.64,-0.10,-3.63,40.78,35.67,124130.0,-11452.0,304390.0,-15.14,56.0,X-LC,4.13,4.0,-0.09,2.18,9.73,X40N,ATH,FMCG
43,ITC,452.00,-0.02,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.74,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.88,-37.71,113.33,32.89,54505.0,-29112.0,48094.0,-693.69,53.0,L-MC,5.96,259.0,-0.53,0.34,32.45,XR,NTT,BANKS
6,ASIANTILES,137.00,1.97,-16.88,118.57,81.67,92454.0,-15836.0,77974.0,7005.56,45.0,L-SC,12.36,271.0,-0.17,0.56,51.00,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.08,-2.11,24.68,22.05,23671.0,-2070.0,95910.0,-15.54,54.0,H-SC,7.25,164.0,-0.09,0.69,39.41,XR,ATH,FINANCE
13,BSOFT,831.70,-1.53,-22.39,94.28,50.79,102922.0,-31487.0,109166.0,-0.27,64.0,H-SC,6.34,151.0,-0.31,0.78,27.10,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.83,-13.29,15.35,0.02,23233.0,-23205.0,151354.0,131.03,49.0,M-SC,14.79,234.0,-1.00,1.08,31.65,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-1.53,-22.39,94.28,50.79,102922.0,-31487.0,109166.0,-0.27,64.0,H-SC,6.34,151.0,-0.31,0.78,27.10,XR,ATH,IT
84,WIPRO,420.0,-0.97,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.88,XR,NTT,IT
41,INFY,1972.0,-0.97,8.32,23.93,34.24,81105.0,26038.0,338926.0,-15.65,59.0,X-LC,4.00,2.0,0.32,2.43,17.29,X40,NTT,IT
1,ABB,7934.0,-0.62,-3.64,51.07,45.57,128745.0,-9523.0,252096.0,-38.07,57.0,H-MC,4.55,120.0,-0.07,1.80,7.27,AR,NTT,ELECTRICAL
27,GLAXO,3466.2,0.73,4.25,33.47,39.15,69537.0,8476.0,207760.0,-16.20,55.0,X-MC,10.68,60.0,0.12,1.49,34.49,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.22
1,25,44.83
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.95
MC    31.01
LC    24.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.69
X40      22.98
X40N     13.07
XR        9.77
XY25      9.10
AR        9.06
OX40N     7.58
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.53
H-SC    23.78
X-LC    20.61
M-SC    11.70
X-SC     8.05
H-MC     4.82
L-SC     1.42
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.41,-7.35,41.86
IT,12.99,-15.81,74.04
FINANCE,9.48,-20.39,72.75
MISC,7.27,-27.88,80.20
ELECTRICAL,6.02,-11.51,52.36
PAINTS,5.84,-13.03,29.82
INSURANCE,4.75,-2.19,37.16
PHARMA,4.14,-1.75,33.78
AUTO,2.80,-33.77,80.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3193791.0,21
AR,1307047.0,10
XR,1295001.0,13
X40,1013961.0,14
X40N,859271.0,9
OX40N,731541.0,10
XY25,370727.0,6
SR,281921.0,2
MH,77026.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3655638.0,25
M-SC,1424290.0,15
X-MC,1402967.0,16
X-LC,846051.0,11
X-SC,793756.0,8
H-MC,404796.0,3
L-SC,262389.0,3
M-LC,111748.0,1
H-LC,76168.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1281412.0      6
           AR         894278.0      5
M-SC       XY24       821125.0      6
H-SC       XR         778588.0      7
X-MC       X40        489494.0      7
           XY24       402822.0      3
H-SC       OX40N      342413.0      4
X-LC       X40        336419.0      5
X-MC       X40N       333596.0      4
X-SC       X40N       308644.0      3
M-SC       OX40N      304019.0      5
X-SC       XY24       297064.0      3
H-SC       SR         281921.0      2
X-LC       X40N       217031.0      2
H-MC       AR         210335.0      2
X-LC       XY24       196907.0      2
H-MC       XY24       194461.0      1
X-SC       X40        188048.0      2
L-SC       XR         177280.0      2
X-MC       XY25       177055.0      2
M-SC       XR         172880.0      2
           AR         126266.0      2
M-LC       XR         111748.0      1
X-LC       XY25        95694.0      2
L-SC       OX40N       85109.0      1
H-SC       MH          77026.0      1
H-LC       AR          76168.0      1
M-MC       XY25        60838.0      1
L-MC       XR          54505.0      1
L-LC       XY25        37140.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
